Window Functions

  

Calculation against a subset of the rows. 

Widow: subset of rows. 

  

Drawback of group by is that you lose the details. The one for subquery is that you have to repeat all the subquery as well as the underlying query. Window function can elegantly overcome these drawbacks.

It also has ability to define order.

Proposition: Compute the running-total values for each employee and month

In [1]:
--Creating a view to generate the table grouped by employeeId and ordermonth.
USE Northwinds2022TSQLV7;
alter VIEW Sales.empOrders
AS
SELECT o.EmployeeId,
	   dateadd (MONTH, datediff (MONTH, 0, o.OrderDate), 0) AS ordermonth,--DATEADD(datepart, number, date)
	   SUM(od.Quantity) AS qty,
	   cast (sum (od.quantity * od.UnitPrice * (1 - od.DiscountPercentage)) AS NUMERIC (12,2)) AS val   
FROM Sales.[Order] AS [o]
	INNER JOIN sales.OrderDetail AS od
		ON od.OrderId = o.OrderId
GROUP BY o.EmployeeId, dateadd (MONTH, datediff (MONTH, 0, o.OrderDate), 0)
GO

SELECT * FROM sales.EmpOrders AS veo --see what the view generates

: Msg 208, Level 16, State 1, Procedure empOrders, Line 8
Invalid object name 'Sales.Order'.

: Msg 208, Level 16, State 1, Line 14
Invalid object name 'sales.EmpOrders'.

Total execution time: 00:00:00.015

In [ ]:
USE Northwinds2022TSQLV7
SELECT veo.EmployeeId,
       veo.ordermonth,
       veo.qty,
	   SUM(qty) OVER (PARTITION BY veo.EmployeeId
					ORDER BY veo.ordermonth
					ROWS BETWEEN UNBOUNDED PRECEDING --no low boundary point
						AND CURRENT ROW) AS runqty --extends until the current row
 
FROM Sales.empOrders AS veo

--window functions are only allowed in the SELECT and ORDER BY. If you need it earliler like in WHERE phase, use TABLE EXPRESSION.

Ranking window functions

  

Proposition: Use ranking functions agains Sales.OrderValues. Use val for ORDER BY.

In [1]:
USE Northwinds2022TSQLV7
SELECT OrderId, CustomerId, Freight,
	 row_number() OVER (ORDER BY freight) AS rownum,
	 RANK() OVER (ORDER BY Freight) AS [rank],
	 DENSE_RANK() OVER (ORDER BY freight) AS [dense_rank],
	 NTILE(100) OVER (ORDER BY freight) AS [ntile]
FROM sales.[ORDER]

--row_number() can be nondeterministic as it produces unique values even when there are ties. To make it deterministic, add another column to the ORDER BY. 
--To make it deterministic, use RANK or DENSE_RANK function. 
--RANK: just rank.
--DENSE_RANK: distinct rank
--NTILE: divide the result set into 100. 

(830 rows affected)

Total execution time: 00:00:00.081

OrderId,CustomerId,Freight,rownum,rank,dense_rank,ntile
10972,40,0.02,1,1,1,1
10296,46,0.12,2,2,2,1
10644,88,0.14,3,3,3,1
10509,6,0.15,4,4,4,1
11035,76,0.17,5,5,5,1
10415,36,0.20,6,6,6,1
10969,15,0.21,7,7,7,1
11054,12,0.33,8,8,8,1
10322,58,0.40,9,9,9,1
10371,41,0.45,10,10,10,2


PARTITION OVER

Proposition: Assign row numbers independently for each customer

In [ ]:
USE Northwinds2022TSQLV7
SELECT orderId, o.CustomerId, o.Freight,
	  ROW_NUMBER() OVER (PARTITION BY o.CustomerId ORDER BY o.Freight) AS rownum
FROM sales.[Order] AS o

--if you want distinct rows, use GROUP BY instead of DISTINCT. Had to drop the orderId for it is not contained in the group by.
SELECT o.customerId, o.Freight,
	   ROW_NUMBER() OVER (PARTITION BY o.CustomerId ORDER BY o.Freight) AS rownum
FROM Sales.[Order] AS o
GROUP BY o.customerId, o.Freight

Offset Window Functions

LAG: looks before the current row

LEAD: looks ahead

default is 1.

Proposition: Find out for each order the previous and the next value ordered by orderDate and OrderID.

In [ ]:
USE Northwinds2022TSQLV7
SELECT o.orderId, o.CustomerId, o.Freight,
	  LAG(o.Freight) OVER (PARTITION BY o.CustomerId ORDER BY orderdate, o.OrderId) AS prevval,
	  LEAD(o.Freight) OVER (PARTITION BY o.CustomerId ORDER BY orderdate, o.OrderId) AS nextval
FROM sales.[Order] AS o
ORDER BY o.CustomerId, o.OrderDate, o.OrderId

--You can specify the increment and decrement to be 3 and return value(0) when a row is not found.
--LAG(val, 3, 0)

FIRST\_VALUE, LAST\_VALUE

Proposition: Return the value of the first customer's order and the value of the last customer's order.

In [ ]:
USE Northwinds2022TSQLV7
SELECT o.orderId, o.CustomerId, o.Freight,
	  FIRST_VALUE (o.Freight) OVER (PARTITION BY o.CustomerId ORDER BY orderdate, o.OrderId ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS firstval,
	  LAST_VALUE (o.Freight) OVER (PARTITION BY o.CustomerId ORDER BY orderdate, o.OrderId ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS lastval
FROM sales.[Order] AS o
ORDER BY o.CustomerId, o.OrderDate, o.OrderId


AGGREGATE WINDOW FUNCTIONS: SUM

Proposition: Return each order along with the grand total of all order values as well as the customer total.  

>

In [ ]:
USE Northwinds2022TSQLV7
SELECT o.orderId, o.CustomerId, o.Freight,
	  SUM(o.Freight) OVER () AS totalValue,
	  SUM(o.Freight) OVER (PARTITION BY o.CustomerId) AS custTotalVal
FROM sales.[Order] AS o
ORDER BY o.Freight


APPLICATION OF AGGREGATE WINDOW FUNCTION

Proposition: Return for each row the percentage of the current value oft of the grand total, as well as out of the customer total

In [ ]:
USE Northwinds2022TSQLV7
SELECT o.orderId, o.CustomerId, o.Freight,
	  FORMAT(o.Freight/SUM(o.Freight) OVER (),'P') AS ptall,
	  format (o.Freight/SUM(o.Freight) OVER (PARTITION BY o.CustomerId),'P') AS pctcust
FROM sales.[Order] AS o
ORDER BY o.Freight


Proposition: Calculate running total of each employee order by month

In [ ]:
USE Northwinds2022TSQLV7
SELECT veo.EmployeeId,
       veo.ordermonth,
       veo.qty,
	   SUM(qty) OVER (PARTITION BY veo.EmployeeId
					ORDER BY veo.ordermonth
					ROWS BETWEEN UNBOUNDED PRECEDING --no low boundary point
						AND CURRENT ROW) AS runqty --extends until the current row
 
FROM Sales.empOrders AS veo

Pivoting data

Rotates data from a state of rows to a state of column. Used for reporting.

Proposition: Return total order quantity for each employee and customer.

In [ ]:
USE Northwinds2022TSQLV7
DROP TABLE IF EXISTS dbo.OrdersExample;
GO

CREATE TABLE dbo.OrdersExample
(
  orderid   INT        NOT NULL,
  orderdate DATE       NOT NULL,
  empid     INT        NOT NULL,
  custid    VARCHAR(5) NOT NULL,
  qty       INT        NOT NULL,
  CONSTRAINT PK_OrdersExample PRIMARY KEY(orderid)
);

INSERT INTO dbo.OrdersExample(orderid, orderdate, empid, custid, qty)
VALUES
  (30001, '20140802', 3, 'A', 10),
  (10001, '20141224', 2, 'A', 12),
  (10005, '20141224', 1, 'B', 20),
  (40001, '20150109', 2, 'A', 40),
  (10006, '20150118', 1, 'C', 14),
  (20001, '20150212', 2, 'B', 12),
  (40005, '20160212', 3, 'A', 10),
  (20002, '20160216', 1, 'C', 20),
  (30003, '20160418', 2, 'B', 15),
  (30004, '20140418', 3, 'C', 22),
  (30007, '20160907', 3, 'D', 30);

SELECT * FROM dbo.OrdersExample;
SELECT oe.empid,
       oe.custid,
       sum (oe.qty) 
FROM dbo.OrdersExample AS oe
GROUP BY oe.empid, oe.custid;

--You need to produce a single row in the result for each unique employeeId. "Spread" the quantities by customerId. 


Pivoting with a grouped query

The grouping phase is achieved with a GROUP BY clause.

The spreading phase is achieved in the SELECT caluse with a CASE expression for each target column.

Proposition: Return total order quantity for each employee and customer.

In [ ]:
USE Northwinds2022TSQLV7
SELECT oe.empid,
	 SUM(CASE WHEN oe.custid = 'A' THEN qty END) AS A, 
	 SUM(CASE WHEN oe.custid = 'B' THEN qty END) AS B,
	 SUM(CASE WHEN oe.custid = 'C' THEN qty END) AS C,
	 SUM(CASE WHEN oe.custid = 'D' THEN qty END) AS D
FROM dbo.OrdersExample AS oe
GROUP BY oe.empid

Pivoting with the Pivot Operator

PIVOT operator figures out the grouping elements implicitly by elmination.

Proposition: Return total order quantity for each employee and customer.

In [ ]:
USE Northwinds2022TSQLV7
SELECT empid, A, B, C, D 
FROM (SELECT empid, custid, qty
	  FROM dbo.OrdersExample AS oe) AS D
	  PIVOT (SUM(qty) FOR custid IN (A, B, C, D)) AS P;

--It is good practice to pivot from a table expression because you do not want to group by irrelevant variables. 

Proposition: Return total order quantity for each customer.

In [ ]:
USE Northwinds2022TSQLV7
SELECT custid, [1], [2], [3]
FROM (SELECT empid, custid, qty
	  FROM dbo.OrdersExample AS oe) AS D
	  PIVOT (SUM(qty) FOR empid IN ([1], [2], [3])) AS P;

Unpivoting data

It is a technique that rotates data from a state of columns to a state of rows. A common use case is to unpivot data you imported from a spreadsheet into the database for easier manipulation. Let's create a pivot table so we can start unpivoting.

In [ ]:
--Creating a table to store the pivoted information from dbo.OrdersExample
CREATE TABLE dbo.EmpCustOrders
(
  empid INT NOT NULL
    CONSTRAINT PK_EmpCustOrders PRIMARY KEY,
  A VARCHAR(5) NULL,
  B VARCHAR(5) NULL,
  C VARCHAR(5) NULL,
  D VARCHAR(5) NULL
);

--Inserting pivot table information
INSERT INTO dbo.EmpCustOrders(empid, A, B, C, D)
  SELECT empid, A, B, C, D
  FROM (SELECT empid, custid, qty
        FROM dbo.OrdersExample AS oe) AS D
    PIVOT(SUM(qty) FOR custid IN(A, B, C, D)) AS P;

--result of pivot table
SELECT * FROM dbo.EmpCustOrders;
--Employee-customer combination that had no intersecting order activity is represented by NULLs.

Unpivoting with the APPLY operator

Unpivoting involves three logical processing phases: producing copies, extracting values, and eliminating irrelevant rows. Use CROSS APPLY without the third step when you need to keep the NULLs. Otherwise, it is simpler to use UNPIVOT.

In [ ]:
--1. Produce copies.
SELECT * 
FROM dbo.EmpCustOrders
	CROSS JOIN (VALUES ('A'), ('B'), ('C'), ('D')) AS C (custid);

--the VALUES clause defines a set of four rows, each with a single customerID value. The code defines a derived table called C based on this clause and names the only column in it custid.
--The code then applies a cross join between EmpCustOrders and C.

--2.Extract values
SELECT empid, custid, qty 
FROM dbo.EmpCustOrders AS eco
	CROSS APPLY (VALUES ('A', A), ('B',B), ('C',C), ('D',D)) AS C(custid, qty) -- USE CROSS APPLY instead of CROSS JOIN because custid and qty are not available with CROSS JOIN.
WHERE qty IS NOT NULL;--3. Eliminate irrelavant rows: removes the rows with NULLs.

Unpivoting with the UNPIVOT operator

Unpivoting data involves producing two result columns from any number of source columns -- one to hold the source column names as strings and another to hold the source column values.

In [ ]:
SELECT empid, custid, qty
FROM dbo.EmpCustOrders AS eco
	UNPIVOT (qty FOR custid IN (A,B,C,D)) AS U;

Grouping Sets

Set: there is no significance to the order in which you specify.

Proposition: Return order information grouped by combination of empid, custid, and the last one should be the for the entire orders.

In [ ]:
USE Northwinds2022TSQLV7
SELECT empid, custid, SUM(qty) AS sumqty
FROM dbo.OrdersExample 
GROUP BY empid, custid;

SELECT empid, SUM(qty) AS sumqty
FROM dbo.OrdersExample 
GROUP BY empid;

SELECT custid, SUM(qty) AS sumqty
FROM dbo.OrdersExample 
GROUP BY custid;

SELECT SUM(qty) AS sumqty
FROM dbo.OrdersExample 

--GROUP SETS subclase. You can achieve the same results using a shortened syntax. The following returns the result in one table set.
SELECT empid, custid, SUM(qty) AS sumqty
FROM dbo.OrdersExample AS oe
GROUP BY
  GROUPING SETS
  (
    (empid, custid),
    (empid),
    (custid),
    ()
  );

The CUBE subclause

Proposition: Return a power set of GROUPING SETS. Set of all subsets of elements that can be produced from a particular set.

CUBE (a,b,c) is equivalent to

GROUPING SETS ((a,b,c), (a,b),(a,c),(b,c),(a), (b), (v), ())

In [ ]:
USE Northwinds2022TSQLV7
SELECT empid, custid, SUM(qty) AS sumqty
FROM dbo.OrdersExample
GROUP BY CUBE(empid, custid);

The ROLLUP subclause

It is like the CUBE subclase except in returns only four based on the hierarchy. ASsuming hierarchy a\>b\>c. 

ROLLUP (a,b,c) is equievalent to GROUPING SETS ((a,b,c),(a,b), (a), ())

In [ ]:
USE Northwinds2022TSQLV7
SELECT 
  YEAR(orderdate) AS orderyear,
  MONTH(orderdate) AS ordermonth,
  DAY(orderdate) AS orderday,
  SUM(qty) AS sumqty
FROM dbo.OrdersExample
GROUP BY ROLLUP(YEAR(orderdate), MONTH(orderdate), DAY(orderdate));

The GROUPING 

If a group column allows NULLs in the table, you cannot tell for sure whether a NULL in the result set originated from the data or is a placeholder for a nonparticiating member in a grouping set. GROUPING function accepts a name of a column and returns 0 if it is a member of the current grouping set (a detail element) and 1 otherwise (an agrgregate element).

In [ ]:
USE Northwinds2022TSQLV7
SELECT 
	GROUPING (empid) AS grpemp,
	GROUPING (custid) AS grpcust,
	empid, custid, SUM(qty) AS sumqty
FROM dbo.OrdersExample
GROUP BY CUBE (empid, custid);



GROUPING\_ID

Instead of GROUPING function for each grouping element as in the previous query, you can call the GROUPING\_ID function once and provide it with all grouping elements as input.

In [ ]:
USE Northwinds2022TSQLV7
SELECT
  GROUPING_ID(empid, custid) AS groupingset,
  empid, custid, SUM(qty) AS sumqty
FROM dbo.OrdersExample
GROUP BY CUBE(empid, custid);